In [ ]:
!nvidia-smi


Sat Jan 17 00:31:19 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
%%writefile device_info.cu
#include <stdio.h>
#include <cuda_runtime.h>

int main() {
    int device;
    cudaGetDevice(&device);

    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, device);

    int driverVersion = 0, runtimeVersion = 0;
    cudaDriverGetVersion(&driverVersion);
    cudaRuntimeGetVersion(&runtimeVersion);

    printf("Device Name: %s\n", prop.name);
    printf("Driver Version: %d\n", driverVersion);
    printf("Runtime Version: %d\n", runtimeVersion);
    printf("Compute Capability: %d.%d\n", prop.major, prop.minor);
    printf("Global Memory: %lu bytes\n", prop.totalGlobalMem);

    return 0;
}


Overwriting device_info.cu


In [ ]:
!nvcc -arch=sm_75 device_info.cu -o device_info


In [ ]:
!./device_info


Device Name: Tesla T4
Driver Version: 12040
Runtime Version: 12050
Compute Capability: 7.5
Global Memory: 15828320256 bytes


In [ ]:
%%writefile hello.cu
#include <stdio.h>
#include <cuda_runtime.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
            return 1; \
        } \
    } while (0)

__global__ void gpuHello() {
    printf("Hello World from GPU\n");
}

int main() {
    printf("Hello World from CPU\n");
    gpuHello<<<1, 1>>>();
    CHECK_CUDA(cudaGetLastError());
    CHECK_CUDA(cudaDeviceSynchronize());
    return 0;
}


Overwriting hello.cu


In [ ]:
!nvcc -arch=sm_75 hello.cu -o hello
!./hello


Hello World from CPU
Hello World from GPU


In [ ]:
%%writefile name_n_times.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define CHECK_CUDA(call) \
    do { \
        cudaError_t err = call; \
        if (err != cudaSuccess) { \
            fprintf(stderr, "CUDA error at %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(err)); \
            return 1; \
        } \
    } while (0)

__global__ void gpuPrint(char *name, int n) {
    int idx = threadIdx.x;
    if (idx < n) {
        printf("GPU: %s\n", name);
    }
}

int main(int argc, char **argv) {
    int n = 4;
    if (argc > 1) {
        n = atoi(argv[1]);
    } else {
        printf("No value provided; defaulting to %d prints.\n", n);
    }

    if (n < 1) {
        n = 1;
    }

    char name[] = "Saksham Singh Rathore";

    for (int i = 0; i < n; i++) {
        printf("CPU: %s\n", name);
    }

    char *d_name;
    CHECK_CUDA(cudaMalloc(&d_name, sizeof(name)));
    CHECK_CUDA(cudaMemcpy(d_name, name, sizeof(name), cudaMemcpyHostToDevice));

    int threads = n > 1024 ? 1024 : n;
    gpuPrint<<<1, threads>>>(d_name, n);
    CHECK_CUDA(cudaGetLastError());
    CHECK_CUDA(cudaDeviceSynchronize());

    CHECK_CUDA(cudaFree(d_name));
    return 0;
}


Overwriting name_n_times.cu


In [ ]:
!nvcc -arch=sm_75 name_n_times.cu -o name_n_times
!./name_n_times 5


CPU: Saksham Singh Rathore
CPU: Saksham Singh Rathore
CPU: Saksham Singh Rathore
CPU: Saksham Singh Rathore
CPU: Saksham Singh Rathore
GPU: Saksham Singh Rathore
GPU: Saksham Singh Rathore
GPU: Saksham Singh Rathore
GPU: Saksham Singh Rathore
GPU: Saksham Singh Rathore
